In [1]:
import os

In [2]:
import pandas as pd

In [3]:
import pyarrow

In [4]:
# time utilities for time-based identifier
import time

In [5]:
from datetime import datetime, timedelta

In [6]:
from bokeh.plotting import figure, show, output_file, save
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [7]:
# Pandas can retrieve data from Parquet by column name
mird_columns = ['timestamp',
                'Van', 'Vbn', 'Vcn', 'Vav',
                'ia', 'ib', 'ic', 'iav',
                'kw', 'kvar', 'kwan', 'kwbn', 'kwcn', 'kvaran', 'kvarbn', 'kvarcn',
                 'f', 'fp',
                'thdvan', 'thdvbn', 'thdvcn', 'thdia', 'thdib', 'thdic',
                'desbV', 'desbI',
                'kwhE', 'kwhR', 'kvarhDel', 'kvarhrec', 'kvarhq3', 'kvarhq4']

In [8]:
SOURCE_PARQUET_PATH = '/home/developer/On_Premises/MIRD_ROOT/data/raw'

In [9]:
resolution = 'hourly'

In [10]:
device = 'CPE04115'

In [11]:
path = '{}/{}/{}.parquet'.format(SOURCE_PARQUET_PATH, resolution, device)
available_dates = os.listdir(path=path)
available_dates.sort()

start_date, end_date = available_dates[0], available_dates[-1]

print('Data is available for {} available dates of {}, from {} to {}.'.format(len(available_dates),
                                                                              device,
                                                                              start_date,
                                                                              end_date))

Data is available for 1343 available dates of CPE04115, from 2016-01-01 to 2019-11-07.


In [12]:
# now mark the selected date interval for the analysis
# data is lost for almost two months, starting 2018-08-10
# trim the data from 2016JAN to 2018JUL, for 31 complete months

In [13]:
# manually redefine the analysis interval
start_date, end_date = '2016-01-01', '2018-07-31'

In [14]:
# get datetimes for start and end dates
start_datetime = datetime.strptime(start_date, '%Y-%m-%d')
end_datetime = datetime.strptime(end_date, '%Y-%m-%d')

In [15]:
# how long is the datetime range for the device?
datetime_range = [start_datetime + timedelta(days=x) for x in range(0, (end_datetime - start_datetime).days + 1)]

In [16]:
print('Data is required for {} valid dates between {} and {}.'.format(len(datetime_range),
                                                                     start_date,
                                                                     end_date))

Data is required for 943 valid dates between 2016-01-01 and 2018-07-31.


In [17]:
# get a list with the required dates to complete the interval
required_dates = [str(datetime)[:10] for datetime in datetime_range]
required_dates.sort()

In [18]:
# is there any valid date missing in the acquired interval?
missing_dates = [date for date in required_dates if date not in available_dates]
missing_dates.sort()
print('Found {} required dates missing in the available dataset.'.format(len(missing_dates)))

Found 0 required dates missing in the available dataset.


In [19]:
base_df = pd.DataFrame(columns=mird_columns)

In [20]:
for date in required_dates:
    path = '{}/{}/{}.parquet/{}'.format(SOURCE_PARQUET_PATH, resolution, device, date)
    
    buffer_df = pd.read_parquet(path,
                                columns=mird_columns,
                                engine='pyarrow')

    base_df = base_df.append(buffer_df, ignore_index=True)
    
# need to change timestamp column from string to datetime
base_df['timestamp'] = pd.to_datetime(base_df['timestamp'])
# sort the data on timestamp because the order might have been lost in the previous operations
base_df = base_df.sort_values(by=['timestamp'])
# re-index data on timestamp column
base_df = base_df.set_index('timestamp')

In [21]:
# verify the base dataframe
base_df

,Van,Vbn,Vcn,Vav,ia,ib,ic,iav,kw,kvar,...,thdib,thdic,desbV,desbI,kwhE,kwhR,kvarhDel,kvarhrec,kvarhq3,kvarhq4
timestamp,,,,,,,,,,,,,,,,,,,,,
2016-01-01 00:00:00,7991.550000,7995.656667,7953.081667,13805.566667,102.378333,75.071367,86.018033,87.822500,2089.078333,165.990167,...,0.048200,0.060117,0.000200,0.016367,173.666667,0.0,13.333333,0.0,0.0,0.0
2016-01-01 01:00:00,7962.071667,7967.080000,7924.573333,13755.666667,96.282683,71.170117,81.418733,82.957167,1966.555000,147.476000,...,0.047983,0.060733,0.000200,0.015900,163.333333,0.0,12.166667,0.0,0.0,0.0
2016-01-01 02:00:00,7960.150000,7965.326667,7922.908333,13752.583333,90.951667,68.328600,76.391567,78.557283,1861.533333,139.629000,...,0.048767,0.061917,0.000200,0.015567,155.166667,0.0,11.833333,0.0,0.0,0.0
2016-01-01 03:00:00,7957.391667,7962.635000,7922.533333,13749.233333,84.878417,64.244167,74.124150,74.415567,1763.545000,122.566500,...,0.049867,0.062117,0.000200,0.014183,146.333333,0.0,10.000000,0.0,0.0,0.0
2016-01-01 04:00:00,7965.773333,7969.850000,7933.076667,13764.266667,79.699933,61.212633,70.431350,70.447983,1671.995000,110.991333,...,0.045950,0.059667,0.000183,0.013367,138.666667,0.0,9.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-07-31 19:00:00,7861.456667,7856.568333,7829.065000,13578.833333,134.919000,116.542500,124.206667,125.222500,2902.200000,487.758333,...,3.648333,4.278333,0.100000,7.746467,241.000000,0.0,40.000000,0.0,0.0,0.0
2018-07-31 20:00:00,7855.471667,7850.665000,7820.493333,13567.016667,134.807667,115.811500,125.180500,125.266667,2912.063333,404.165167,...,4.013333,4.640000,0.116667,7.768217,243.666667,0.0,33.666667,0.0,0.0,0.0
2018-07-31 21:00:00,7826.050000,7822.853333,7791.426667,13517.250000,139.743667,117.616333,128.172333,128.510667,2983.391667,360.991833,...,4.083333,4.708333,0.133333,8.853917,248.166667,0.0,30.000000,0.0,0.0,0.0


In [22]:
# save a version of the base dataframe in CSV format to test TFX components
# (to developer home path)

In [23]:
# base_df.to_csv('/home/developer/CPE04115_H.csv')

In [24]:
percentage = 0.995
slack = 1.25
ceil_kw = base_df[['kw']].quantile(percentage).kw*slack

In [25]:
fig_kw = figure(
    x_axis_type='datetime',
    y_range=(0., ceil_kw),
    plot_width=960,
    plot_height=400,
    title='Active Power (hourly) for {}.'.format(device))

fig_kw.grid.grid_line_alpha=0.3

fig_kw.xaxis.axis_label = 'Date'
fig_kw.yaxis.axis_label = 'Active Power [W]'

fig_kw.line(base_df.index, base_df.kw, color='#A6CEE3', legend_label='kw')

# uncomment the following two lines to save plot
# output_file('/home/developer/gcp/cbidmltsf/datasets/cfe/{}_H_kw.html'.format(device))
# save(fig_kw)

# uncomment the following line to display plot
show(fig_kw)

In [26]:
# there are no missing values
# identify outliers by Z-score

In [27]:
import numpy as np

In [28]:
from scipy.stats import zscore

In [29]:
preprocessed_df = base_df.copy()

In [30]:
z_threshold = [-2.0, 2.6]
low_outliers = list(zscore(base_df.kw) < z_threshold[0])
high_outliers = list(zscore(base_df.kw) > z_threshold[1])

In [31]:
outliers_list = [x or y for x, y in zip(low_outliers, high_outliers)]

In [32]:
print('Found {} outliers with absolute Z-score outside {} in {} lectures.'.format(sum(outliers_list),
                                                                                       z_threshold,
                                                                                       base_df.kw.count()))

Found 59 outliers with absolute Z-score outside [-2.0, 2.6] in 22629 lectures.


In [33]:
# a new dataframe with outliers set to None
preprocessed_df.kw[outliers_list] = None

In [34]:
fig_kw = figure(
    x_axis_type='datetime',
    y_range=(0., ceil_kw),
    plot_width=960,
    plot_height=400,
    title='Active Power (hourly) for {}.'.format(device))

fig_kw.grid.grid_line_alpha=0.3

fig_kw.xaxis.axis_label = 'Date'
fig_kw.yaxis.axis_label = 'Active Power [W]'

fig_kw.line(base_df.index, preprocessed_df.kw, color='#A6CEE3', legend_label='kw')

# uncomment the following two lines to save plot
# output_file('/home/developer/gcp/cbidmltsf/datasets/cfe/{}_H_kw.html'.format(device))
# save(fig_kw)

# uncomment the following line to display plot
show(fig_kw)

In [35]:
# simple correction of outliers
# update NaN values to
# the immediate last week value
# (or, maybe, the average of the last n week-values)

In [36]:
# a list with datetimes where kw is None
dates_to_fill = preprocessed_df.index[outliers_list]

In [37]:
# traverse all dates with a NaN in the variable of interest (kw)
for date in dates_to_fill:
    # get the timestamp for a week before
    date_minus_one_week = date - timedelta(days=7)
    # update the missing value to the one in the previous week, if the latter exists
    if preprocessed_df.loc[date_minus_one_week].kw is not None:
        preprocessed_df.loc[date].kw = preprocessed_df.loc[date_minus_one_week].kw

In [38]:
fig_kw = figure(
    x_axis_type='datetime',
    y_range=(0., ceil_kw),
    plot_width=960,
    plot_height=400,
    title='Active Power (hourly) for {}.'.format(device))

fig_kw.grid.grid_line_alpha=0.3

fig_kw.xaxis.axis_label = 'Date'
fig_kw.yaxis.axis_label = 'Active Power [W]'

fig_kw.line(base_df.index, preprocessed_df.kw, color='#A6CEE3', legend_label='kw')

# uncomment the following two lines to save plot
# output_file('/home/developer/gcp/cbidmltsf/datasets/cfe/{}_H_kw.html'.format(device))
# save(fig_kw)

# uncomment the following line to display plot
show(fig_kw)

In [39]:
# now, save the resulting time series as it is the base for further work on forecasting
# save the time series only, not the complete dataframe, as other variables have not been preprocessed
# Pandas pickle or Parquet archive? (answer: use the format required to produce SLDBs)

In [40]:
# persist the preprocessed time series to Pandas pickle
# scale it first, for data securiryt, because it will be persisted to the cloud

In [41]:
# get the time series as a copy of the corresponding dataframe column
time_series_kw = preprocessed_df.kw.copy()

In [49]:
time_series_kw.describe()

count    22629.000000
mean      2430.084212
std        455.420271
min       1498.945000
25%       2020.803333
50%       2471.595000
75%       2793.378333
max       3650.228333
Name: kw, dtype: float64

In [51]:
time_series_kw

timestamp
2016-01-01 00:00:00    2089.078333
2016-01-01 01:00:00    1966.555000
2016-01-01 02:00:00    1861.533333
2016-01-01 03:00:00    1763.545000
2016-01-01 04:00:00    1671.995000
                          ...     
2018-07-31 19:00:00    2902.200000
2018-07-31 20:00:00    2912.063333
2018-07-31 21:00:00    2983.391667
2018-07-31 22:00:00    2810.393333
2018-07-31 23:00:00    2548.693333
Name: kw, Length: 22629, dtype: float64

In [43]:
# scale datasets to improve neural networks performance
from sklearn.preprocessing import MinMaxScaler

In [44]:
# scaler persistence
import joblib

In [45]:
# get a scaler for normalization
scaler = MinMaxScaler(feature_range=(0, 1))

In [46]:
# build the scaled time series
time_series_kw_scaled = scaler.fit_transform(np.array(time_series_kw).reshape(-1, 1))

In [47]:
time_series_kw_scaled.shape

(22629, 1)

In [48]:
# build a string identifier for the time series and its directory inside timeseries/
# add a time-based suffix to manage different versions of the same time series
identifier = '{}_{}_{}_{}'.format(device,
                                  resolution[0].upper(),
                                  'kw',
                                  time.strftime('%Y%m%d%H%M%S'))

In [47]:
# build the time series directory
time_series_folder = '/home/developer/gcp/cbidmltsf/timeseries/{}'.format(identifier)

In [48]:
try:
    os.mkdir(time_series_folder)
    print('Directory {} was created.'.format(time_series_folder))
except FileExistsError:
    print('Error: directory {} already exists.'.format(time_series_folder))

Directory /home/developer/gcp/cbidmltsf/timeseries/CPE04115_H_kw_20210211125817 was created.


In [49]:
# persist fitted scaler to timeseries/identifier/
scaler_filename = '{}/scaler.save'.format(time_series_folder)

In [50]:
joblib.dump(scaler, scaler_filename)

['/home/developer/gcp/cbidmltsf/timeseries/CPE04115_H_kw_20210211125817/scaler.save']

In [51]:
# the scaled time series is now a NumPy array, with only values for the variable
# the array does not contain timestamps
# need to build a new Pandas time series from the scaled one
# to add timestamp before persisting it to disk

In [52]:
time_series_kw_scaled_df = pd.DataFrame(data=time_series_kw_scaled,
                                        columns=['kw_scaled'],
                                        index=time_series_kw.index)

In [53]:
time_series_kw_scaled_df

,kw_scaled
timestamp,
2016-01-01 00:00:00,0.274317
2016-01-01 01:00:00,0.217363
2016-01-01 02:00:00,0.168545
2016-01-01 03:00:00,0.122996
2016-01-01 04:00:00,0.080440
...,...
2018-07-31 19:00:00,0.652287
2018-07-31 20:00:00,0.656872
2018-07-31 21:00:00,0.690028


In [54]:
fig_kw = figure(
    x_axis_type='datetime',
    y_range=(0., 1.),
    plot_width=960,
    plot_height=400,
    title='Normalized Active Power (hourly) for {}.'.format(device))

fig_kw.grid.grid_line_alpha=0.3

fig_kw.xaxis.axis_label = 'Date'
fig_kw.yaxis.axis_label = 'Normalized Active Power'

fig_kw.line(time_series_kw_scaled_df.index,
            time_series_kw_scaled_df.kw_scaled,
            color='#A6CEE3',
            legend_label='scaled kw')

# uncomment the following two lines to save plot
# output_file('/home/developer/gcp/cbidmltsf/datasets/cfe/{}_H_kw.html'.format(device))
# save(fig_kw)

# uncomment the following line to display plot
show(fig_kw)

In [55]:
# persist the scaled time series as Pandas pickle
pickle_filename = '{}/ts.pkl'.format(time_series_folder)
time_series_kw_scaled_df.to_pickle(pickle_filename)

In [62]:
# save a copy of the time series in CSV to ease ingestion in TFX
# (the component for Parquet ingestion is failing!)
csv_filename = '{}/ts.csv'.format(time_series_folder)
time_series_kw_scaled_df.to_csv(csv_filename)

In [56]:
# and test the persisted time series by opening it to a different variable
test_df = pd.read_pickle(pickle_filename)

In [57]:
test_df.describe()

,kw_scaled
count,22629.000000
mean,0.432830
std,0.211697
min,0.000000
25%,0.242580
50%,0.452125
75%,0.601703
max,1.000000


In [58]:
# time series specs have to be persisted as a JSON file
ts = {
    'device': 'CPE04115',
    'resolution': 'hourly',
    'variable': 'kw',
    'start_timestamp': '2016-01-01 00:00:00',
    'end_timestamp': '2018-07-31 23:00:00',
    'identifier': identifier
}

In [59]:
import json

In [60]:
json_filename = '{}/ts.json'.format(time_series_folder)

In [61]:
# persist time series specs for further use
with open(json_filename, 'w') as outfile:
    json.dump(ts, outfile, indent=4)